### In this notebook, we use cohere command r plus for topic annotation

In [3]:
from rich import print
import json
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
import os
from openai import OpenAI
import cohere
import pandas as pd
from tqdm import tqdm

In [4]:
_ = load_dotenv(find_dotenv())
api_key = os.environ['OPENAI_API_KEY']

# base_url = os.environ['OPENAI_BASE_URL']
co = cohere.Client()

In [5]:
pwd

'/notebooks/ABSA-for-Open-Ended-Qs-in-Education-Surveys/notebooks/label_cleansing'

In [6]:
example1 = "المحتوى أكثر من رائع"

In [7]:
system_message = """
سوف تتلقى مراجعة حول بعض الخدمات التعليمية، ومهمتك هي معرفة الموضوع الذي تتجذث عنه كل مراجعة.
قد تكون المراجعة باللغة الإنجليزية، أو باللغة العربية، أو كليهما.
خيارات الإخراج المسموح بها هي ما يلي:

* محتوى ومعلومات المقرر
* موازنة الجزء العملي مع الجزء النظري
* غير محدد
* سوق العمل
* المتطلبات و المهام و الدرجات
* دكتور المقرر
* الوقت و الجدول
* طريقة تدريس و تقديم المادة
* الجانب البحثي والأوراق البحثية
"""

In [8]:
user_template = """
عليك أن تستخرج الموضوع الخاص بالمراجعة التالية. 

## كيفية استخراج الموضع الخاص بالمراجعة:
* إذا كان نص المراجعة يتحدث عن المقرر الدراسي و المعلومات الواردة في هذا المقرر فإن الموضوع الخاص بهذه المراجعة هو: "محتوى ومعلومات المقرر"

* إذا كان نص المراجعة يتحدث عن الجزء العملي و أهميته و أهمية تناسبه مع الجزء النظري فإن الموضوع الخاص بهذه المراجعة هو: "موازنة الجزء العملي مع الجزء النظري"

* إذا كان نص المراجعة يتحدث عن سوق العمل و مدى ارتباط المنهج بسوق العمل فإن الموضوع الخاص يهذه المراجعة هو: "سوق العمل"

* إذا كان نص المراجعة يتحدث عن الأشياء المطلوب من الطلاب أن يقوموا بها أو عن درجات الطلاب فإن الموضوع الخاص بهذه المراجعة هو: "المتطلبات و المهام و الدرجات"

* إذا كان نص المراجعة يتحدث عن الدكتور الذي يشرح المادة فإن الموضوع الخاص بهذه المراجعة هو: "دكتور المقرر"

* إذا كان نص المراجعة يتحدث عن الخط الزمني لدراسة المادة أو عن جدول تدريس المادة و جدول الإمتحانات و التسليمات فإن الموضوع الخاص بهذه المراجعة هو: "الوقت و الجدول"

* إذا كان نص المراجعة يتحدث عن الطريقة التي يتم بها تقديم المادة للطلاب أو عن الطريقة التي يتم بها تدريس المادة للطلاب فإن الموضوع الخاص بهذه المراجعة هو: "طريقة تدريس و تقديم المادة"

* إذا كان نص المراجعة يتحدث عن الجانب البحثي للمادة أو عن الأوراق البحثية فإن الموضوع الخاص بهذه المراجعة هو: "الجانب البحثي و الأوراق البحثية"

* إذا لم يحتو نص المراجعة على أي من الموضوعات السابقة فإن موضوع هذه المراحعة هو: "غير محدد"

## ملحوظة: إذا وجدت أكثر من موضوع داخل المراجعة الواحدة يجب عليك استخراج الموضوع الموجود بكثرة داخل هذه المراجعة


"
المراجعة:
###
{}
###

## يجب عليك أن تكتب اسم الموضوع المستخرج فقط
output: 
"""

In [9]:
def format_user_message(template, text):
    return template.format(text)


In [10]:
def format_one_example(review):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": format_user_message(user_template, review)
            }
        ]
    }

In [11]:
print(format_one_example(example1)['messages'][1]['content'])

عليك أن تستخرج الموضوع الخاص بالمراجعة التالية. 

## كيفية استخراج الموضع الخاص بالمراجعة:
* إذا كان نص المراجعة يتحدث عن المقرر الدراسي و المعلومات الواردة في هذا المقرر فإن الموضوع الخاص بهذه المراجعة هو:
"محتوى ومعلومات المقرر"

* إذا كان نص المراجعة يتحدث عن الجزء العملي و أهميته و أهمية تناسبه مع الجزء النظري فإن الموضوع الخاص بهذه المراجعة
هو: "موازنة الجزء العملي مع الجزء النظري"

* إذا كان نص المراجعة يتحدث عن سوق العمل و مدى ارتباط المنهج بسوق العمل فإن الموضوع الخاص يهذه المراجعة هو: "سوق 
العمل"

* إذا كان نص المراجعة يتحدث عن الأشياء المطلوب من الطلاب أن يقوموا بها أو عن درجات الطلاب فإن الموضوع الخاص بهذه 
المراجعة هو: "المتطلبات و المهام و الدرجات"

* إذا كان نص المراجعة يتحدث عن الدكتور الذي يشرح المادة فإن الموضوع الخاص بهذه المراجعة هو: "دكتور المقرر"

* إذا كان نص المراجعة يتحدث عن الخط الزمني لدراسة المادة أو عن جدول تدريس المادة و جدول الإمتحانات و التسليمات فإن 
الموضوع الخاص بهذه المراجعة هو: "الوقت و الجدول"

* إذا كان نص المراجعة يتحدث عن الطريقة التي يتم بها تقديم المادة للطلاب أو عن الطريقة التي يتم بها تدريس المادة 
للطلاب فإن الموضوع الخاص بهذه المراجعة هو: "طريقة تدريس و تقديم المادة"

* إذا كان نص المراجعة يتحدث عن الجانب البحثي للمادة أو عن الأوراق البحثية فإن الموضوع الخاص بهذه المراجعة هو: 
"الجانب البحثي و الأوراق البحثية"

* إذا لم يحتو نص المراجعة على أي من الموضوعات السابقة فإن موضوع هذه المراحعة هو: "غير محدد"

## ملحوظة: إذا وجدت أكثر من موضوع داخل المراجعة الواحدة يجب عليك استخراج الموضوع الموجود بكثرة داخل هذه المراجعة


"
المراجعة:
###
المحتوى أكثر من رائع
###

## يجب عليك أن تكتب اسم الموضوع المستخرج فقط
output:

In [12]:
# model_name = 'meta-llama/Meta-Llama-3-70B-Instruct'
# model_name = 'meta-llama/Meta-Llama-3-70B-Instruct'
# model_name = 'gpt-4o'
model_name = 'command-r-plus'
def get_completion(review, model_name):
    completion = co.chat(
        model=model_name, preamble=format_one_example(review)['messages'][0]['content'], 
        temperature=0.0, message=format_one_example(review)['messages'][1]['content'],
        chat_history = [{'role': 'SYSTEM', 'message': system_message}]
    )
    
    return completion

In [13]:
def predict_gpt(item, model_name=model_name):
    label = get_completion(item, model_name)
    # result = json.loads(completion)
    # pred_labels = [result.get(token, 'O') for token in item['tokens']]
    # pred_labels = [v for k, v in result.items()]
    # organizations = result['Organizations']
    return label.text

In [14]:
example1 = "the doctor is very helpful"

In [15]:
print(get_completion(example1, model_name))

NonStreamedChatResponse(
    text='دكتور المقرر',
    generation_id='7535a23e-3b1c-4984-a674-11d16b246492',
    citations=None,
    documents=None,
    is_search_required=None,
    search_queries=None,
    search_results=None,
    finish_reason='COMPLETE',
    tool_calls=None,
    chat_history=[
        Message_System(
            message='\nسوف تتلقى مراجعة حول بعض الخدمات التعليمية، ومهمتك هي معرفة الموضوع الذي تتجذث عنه كل 
مراجعة.\nقد تكون المراجعة باللغة الإنجليزية، أو باللغة العربية، أو كليهما.\nخيارات الإخراج المسموح بها هي ما 
يلي:\n\n* محتوى ومعلومات المقرر\n* موازنة الجزء العملي مع الجزء النظري\n* غير محدد\n* سوق العمل\n* المتطلبات و 
المهام و الدرجات\n* دكتور المقرر\n* الوقت و الجدول\n* طريقة تدريس و تقديم المادة\n* الجانب البحثي والأوراق 
البحثية\n',
            tool_calls=None,
            role='SYSTEM'
        ),
        Message_User(
            message='\nعليك أن تستخرج الموضوع الخاص بالمراجعة التالية. \n\n## كيفية استخراج الموضع الخاص 
بالمراجعة:\n* إذا كان نص المراجعة يتحدث عن المقرر الدراسي و المعلومات الواردة في هذا المقرر فإن الموضوع الخاص بهذه 
المراجعة هو: "محتوى ومعلومات المقرر"\n\n* إذا كان نص المراجعة يتحدث عن الجزء العملي و أهميته و أهمية تناسبه مع 
الجزء النظري فإن الموضوع الخاص بهذه المراجعة هو: "موازنة الجزء العملي مع الجزء النظري"\n\n* إذا كان نص المراجعة 
يتحدث عن سوق العمل و مدى ارتباط المنهج بسوق العمل فإن الموضوع الخاص يهذه المراجعة هو: "سوق العمل"\n\n* إذا كان نص 
المراجعة يتحدث عن الأشياء المطلوب من الطلاب أن يقوموا بها أو عن درجات الطلاب فإن الموضوع الخاص بهذه المراجعة هو: 
"المتطلبات و المهام و الدرجات"\n\n* إذا كان نص المراجعة يتحدث عن الدكتور الذي يشرح المادة فإن الموضوع الخاص بهذه 
المراجعة هو: "دكتور المقرر"\n\n* إذا كان نص المراجعة يتحدث عن الخط الزمني لدراسة المادة أو عن جدول تدريس المادة و 
جدول الإمتحانات و التسليمات فإن الموضوع الخاص بهذه المراجعة هو: "الوقت و الجدول"\n\n* إذا كان نص المراجعة يتحدث عن 
الطريقة التي يتم بها تقديم المادة للطلاب أو عن الطريقة التي يتم بها تدريس المادة للطلاب فإن الموضوع الخاص بهذه 
المراجعة هو: "طريقة تدريس و تقديم المادة"\n\n* إذا كان نص المراجعة يتحدث عن الجانب البحثي للمادة أو عن الأوراق 
البحثية فإن الموضوع الخاص بهذه المراجعة هو: "الجانب البحثي و الأوراق البحثية"\n\n* إذا لم يحتو نص المراجعة على أي 
من الموضوعات السابقة فإن موضوع هذه المراحعة هو: "غير محدد"\n\n## ملحوظة: إذا وجدت أكثر من موضوع داخل المراجعة 
الواحدة يجب عليك استخراج الموضوع الموجود بكثرة داخل هذه المراجعة\n\n\n"\nالمراجعة:\n###\nthe doctor is very 
helpful\n###\n\n## يجب عليك أن تكتب اسم الموضوع المستخرج فقط\noutput: \n',
            tool_calls=None,
            role='USER'
        ),
        Message_Chatbot(message='دكتور المقرر', tool_calls=None, role='CHATBOT')
    ],
    prompt=None,
    meta=ApiMeta(
        api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None),
        billed_units=ApiMetaBilledUnits(
            input_tokens=826,
            output_tokens=5,
            search_units=None,
            classifications=None
        ),
        tokens=ApiMetaTokens(input_tokens=836, output_tokens=5),
        warnings=None
    ),
    response_id='d295bce5-0483-4b60-b5d2-57e543ba183a'
)

In [16]:
train_data_path = '../../data/result/train_df.csv'
test_data_path = '../../data/result/test_df.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [17]:
len(['موازنة الجزء العملي مع الجزء النظري' ,'محتوى ومعلومات المقرر',
 'الوقت و الجدول' ,'دكتور المقرر' ,'طريقة تدريس و تقديم المادة', 'غير محدد',
 'سوق العمل' ,'المتطلبات و المهام و الدرجات',
 'الجانب البحثي و الأوراق البحثية'])



9

In [18]:
train_data.head()

,column,text,topic_id,topic_name
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول


In [19]:
tqdm.pandas()

print('Annotate train data...')
train_data['topic_llm'] = train_data['text'].progress_apply(predict_gpt)
print('Annotate test data...')
test_data['topic_llm'] = test_data['text'].progress_apply(predict_gpt)

Annotate train data...

100%|██████████| 951/951 [11:01<00:00,  1.44it/s]


Annotate test data...

100%|██████████| 634/634 [08:03<00:00,  1.31it/s] 


In [20]:
# for e in train_data['text'].sample(50):
#     # print(e)
#     print('topic:\n', predict_gpt(e))
#     print('----'*25)
    

In [21]:
# train_data['sentiment'] = train_data['sentiment'].progress_apply(str.capitalize)
# test_data['sentiment'] = test_data['sentiment'].progress_apply(str.capitalize)

In [22]:
train_data.head()

,column,text,topic_id,topic_name,topic_llm
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,غير محدد
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري


In [38]:
train_data['topic_llm'].nunique()

9

In [31]:
train_data['topic_llm'] = train_data['topic_llm'].apply(lambda x: "الجانب البحثي و الأوراق البحثية" if x == 'الجانب البحثي والأوراق البحثية' else x)
test_data['topic_llm'] = test_data['topic_llm'].apply(lambda x: "الجانب البحثي و الأوراق البحثية" if x == 'الجانب البحثي والأوراق البحثية' else x)
train_data['topic_llm'] = train_data['topic_llm'].apply(lambda x: "غير محدد" if x not in train_data['topic_name'].unique() else x)
test_data['topic_llm'] = test_data['topic_llm'].apply(lambda x: "غير محدد" if x not in train_data['topic_name'].unique() else x)

In [39]:
train_data['topic_llm'].value_counts() / len(train_data)

غير محدد                               0.231335
دكتور المقرر                           0.172450
موازنة الجزء العملي مع الجزء النظري    0.164038
محتوى ومعلومات المقرر                  0.159832
طريقة تدريس و تقديم المادة             0.089380
المتطلبات و المهام و الدرجات           0.087277
الوقت و الجدول                         0.033649
الجانب البحثي و الأوراق البحثية        0.031546
سوق العمل                              0.030494
Name: topic_llm, dtype: float64

In [40]:
test_data['topic_llm'].value_counts() # / len(test_data)

غير محدد                               142
دكتور المقرر                           115
محتوى ومعلومات المقرر                  110
موازنة الجزء العملي مع الجزء النظري     94
طريقة تدريس و تقديم المادة              60
المتطلبات و المهام و الدرجات            52
الوقت و الجدول                          24
الجانب البحثي و الأوراق البحثية         21
سوق العمل                               16
Name: topic_llm, dtype: int64

In [41]:
train_save_path = '../../data/result/train_with_topic_df_llm_command_r_plus.csv'
test_save_path = '../../data/result/test_with_topic_df_llm_command_r_plus.csv'

if os.path.exists(train_save_path):
    print('The path exists!')
else:
    print(f'Saving to {train_save_path}...')
    train_data.to_csv(train_save_path, index=False)
    

if os.path.exists(test_save_path):
    print('The path exists!')
else:
    print(f'Saving to {test_save_path}...')
    test_data.to_csv(test_save_path, index=False)

Saving to ../../data/result/train_with_topic_df_llm_command_r_plus.csv...

Saving to ../../data/result/test_with_topic_df_llm_command_r_plus.csv...